# 아이유팬이 좋아할 만한 다른 아티스트 찾기
---
### 프로젝트 - Movielens 영화 추천 실습

## <span style="color:purple">실습목표</span>

><span style="color:green">**0.** 추천시스템의 개념과 목적을 이해한다.<br>
**1.** Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.<br>
**2.** 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.<br>
**3.** 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다.<br>
**4.** 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.<br>
**5.** 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.</sapn>

## <span style="color:purple">순서</span>
> <span style="color:green">**Step1.** 데이터 준비와 전처리<br>
**Step2.** 분석해 봅시다.<br>
**Step3.** 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.<br>
**Step4.** CSR matrix를 직접 만들어 봅시다.<br>
**Step5.** als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.<br>
**Step6.** 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.<br>
**Step7.** 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.<br>
**Step8.** 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.</sapn>

### 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.
----
>- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. **`MovieLens 1M Dataset`** 사용을 권장합니다.
>- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
>- 별점을 **시청횟수**로 해석해서 생각하겠습니다.
또한 유저가 <span style="color:red">3점 미만</span>으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

**`MovieLens 1M Dataset`** 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

**1) wget으로 데이터 다운로드**
> $ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

**2) 다운받은 데이터를 작업디렉토리로 옮김**
> $ mv ml-1m.zip ~/aiffel/recommendata_iu/data

**3) 작업디렉토리로 이동**
> $ cd ~/aiffel/recommendata_iu/data

**4) 압축 해제**
> $ unzip ml-1m.zip

### <span style="color:green">Step1. 데이터 준비와 전처리</span>
---
Movielens 데이터는 **`rating.dat`** 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.

ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.

movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**Error]** **<span style="color:Orange">UnicodeDecodeError</span>**: 'utf-8' codec can't decode byte 0xe9 in position 3114: invalid continuation byte 발생하여 따로 **`encoding`**을 지정해주니 해결되었습니다.

### <span style="color:green">Step2. 분석해 봅시다.</span>
---
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [7]:
# from scipy.sparse import csr_matrix

mv_title = ratings['movie_id'].nunique()  # 영화 개수
mv_user = ratings['user_id'].nunique()  # 사용자 수

In [8]:
print("ratings에 있는 유니크한 영화 개수 : ", mv_title, "개")
print("ratings에 있는 유니크한 사용자 수 : ", mv_user, "개")

ratings에 있는 유니크한 영화 개수 :  3628 개
ratings에 있는 유니크한 사용자 수 :  6039 개


In [9]:
# 가장 인기 있는 영화 30개(인기순)

mv_total = pd.merge(ratings, movies, how='left')
mv_total

mv_count = mv_total.groupby('title')['user_id'].count()
mv_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### <span style="color:green">Step3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.</span>

재미있게 보았던 인사이드 아웃, 겨울왕국1, 토이스토리2랑 너무 애니매이션만 있는 것 같아서 마블세계관의 시작인 아이언맨과 가장 최근에 본 작은아씨들까지 추가하기로 했습니다.

In [10]:
my_favorite = ['Inside Out (2015)' , 'Frozen (2013)' ,'Iron Man (2008)' ,'Little Women (2019)' ,'Toy Story2 (1999)']

for movie in my_favorite:
    print(movies[movies['title'] == movie])

Empty DataFrame
Columns: [movie_id, title, genre]
Index: []
Empty DataFrame
Columns: [movie_id, title, genre]
Index: []
Empty DataFrame
Columns: [movie_id, title, genre]
Index: []
Empty DataFrame
Columns: [movie_id, title, genre]
Index: []
Empty DataFrame
Columns: [movie_id, title, genre]
Index: []


### <span style="color:green">Step4. CSR matrix를 직접 만들어 봅시다.</span>

In [11]:
using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [12]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data    
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
1000203,6038,1030,3
1000205,6038,986,5
1000206,6038,311,5
1000207,6038,142,4


In [13]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6039x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836478 stored elements in Compressed Sparse Row format>

### <span style="color:green">Step5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.</span>

In [14]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [15]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6039 sparse matrix of type '<class 'numpy.int64'>'
	with 836478 stored elements in Compressed Sparse Column format>

In [16]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### <span style="color:green">Step6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.</span>

### <span style="color:green">Step7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.</span>

### <span style="color:green">Step8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.</span>

# <span style="color:purple">루브릭 평가 기준</span>
---
번호|<center>평가문항</center>|<center>상세기준</center>
---|---|---
1 |<center>**CSR matrix**가 정상적으로 만들어졌다.</center> |</center>사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.</center>
2 |<center>**MF 모델**이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.</center> |</center>사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.</center>
3 |<center>비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.</center> |</center>MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.</center>

# <span style="color:purple">회고</span>
---
2주의 시간이 무척 행복했어요.